In [13]:
import boto3

ImportError: cannot import name 'logger' from partially initialized module 'boto_functions.ec2_functions' (most likely due to a circular import) (/home/avinash/Data-Science/User-Behaviour-Analytics/User-Behaviour-Analytics/boto_functions/ec2_functions.py)

In [2]:
s3_resource = boto3.resource('s3')

In [4]:
bucket = 'test-for-infra-script'

In [5]:
create_bucket(bucket_name=bucket, s3_resource=s3_resource, logger=logger)

{'ResponseMetadata': {'RequestId': 'P3NCJC69SA6KW5JQ',
  'HostId': 'bvg7V/obf0UNfavNkc1i+npPXYrusxCzNK1OXAvyyoql5wT6nSPR6AnKdwRB2XVqgV78oDs3eKQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bvg7V/obf0UNfavNkc1i+npPXYrusxCzNK1OXAvyyoql5wT6nSPR6AnKdwRB2XVqgV78oDs3eKQ=',
   'x-amz-request-id': 'P3NCJC69SA6KW5JQ',
   'date': 'Tue, 19 Jul 2022 07:21:26 GMT',
   'location': 'http://test-for-infra-script.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://test-for-infra-script.s3.amazonaws.com/'}

In [5]:
delete_bucket(bucket_name=bucket, s3_resource=s3_resource, logger=logger)

{'ResponseMetadata': {'RequestId': '1E6MX85V0WG8JSQ2',
  'HostId': 'Rp1Q1WevwcB1U8vFujQT41bYmStpLeg7AZi5VSq/3ESBeO8wJcuqFiN4WaYS7NFrH8h4oUVV00I=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Rp1Q1WevwcB1U8vFujQT41bYmStpLeg7AZi5VSq/3ESBeO8wJcuqFiN4WaYS7NFrH8h4oUVV00I=',
   'x-amz-request-id': '1E6MX85V0WG8JSQ2',
   'date': 'Tue, 19 Jul 2022 07:31:09 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [1]:
import socket
hostname = socket.gethostname()   
IPAddr = socket.gethostbyname(hostname)   
print("Your Computer Name is:" + hostname)   
print("Your Computer IP Address is:" + IPAddr)

Your Computer Name is:MSI
Your Computer IP Address is:127.0.1.1


In [10]:
#get my ip
import subprocess
ip = subprocess.run(['curl', '-s', 'http://whatismyip.akamai.com/'],stderr=subprocess.PIPE, text=True)
print(ip.stderr)

122.161.240.125


In [38]:
import subprocess
def my_ip():
    process = subprocess.Popen(['curl', '-s', 'http://whatismyip.akamai.com/'],stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    ip = process.communicate()
    return ip[0]

In [39]:
ip = my_ip()
ip

'122.161.240.125'

In [3]:
ip[0]

'122.161.240.125'

In [5]:
def attach_ingress(my_ip, security_group, group_id, description):
    cidr = my_ip+'/32'
    response = security_group.authorize_ingress(
        CidrIp=cidr,
        FromPort=23,
        GroupId= group_id,
        IpProtocol='tcp',
        ToPort=123 
    )

def attach_egress(my_ip, security_group):
    """ attaches egress rule, Port is hardcoded, free to edit"""
    cidr = my_ip+'/32'
    response = security_group.authorize_egress(
        IpPermissions=[
        {
            'FromPort': 8080,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': cidr,
                    'Description': 'my_ip'
                },
            ],
            'ToPort': 8080
        },
    ],
    )

In [6]:
ec2 = boto3.resource('ec2')

In [7]:

security_group = ec2.SecurityGroup('sg-0c7f8f5f8b15ee00c')

In [8]:
ec2_security_group_id = security_group.group_id
ec2_security_group_id

'sg-0c7f8f5f8b15ee00c'

In [13]:
from botocore.exceptions import ClientError
def create_instance(ec2, security_group_id, instance_profile_name, instance_type):
    """ creates an EC2 instance for our computing needs
    Hardcoded: instance number max and min values, OS image_id, volumetype, availability zone, 
    """
    try:
        instance = ec2.create_instances(
            BlockDeviceMappings=[
                {
                    'DeviceName': '/dev/sdh',
                    'Ebs': {
                        'DeleteOnTermination': False,
                        'VolumeSize': 8,
                        'VolumeType': 'gp2'
                    },
                },
            ],
            ImageId='ami-08df646e18b182346',
            InstanceType=instance_type,
            MaxCount=1,
            MinCount=1,
            Monitoring={
                'Enabled': False
            },
            SecurityGroupIds=[
                security_group_id
            ],
            IamInstanceProfile={
                'Name': instance_profile_name
            },
            
        )
    except ClientError:
        raise
    else:
        return instance

In [15]:
instance = create_instance(ec2=ec2, security_group_id=ec2_security_group_id, instance_profile_name='ec2-role-project-sr1-19-7-2022-instance-profile', instance_type='t2.large')

ec2.Instance(id='i-065e425bd2e496e5f')

In [29]:
response = instance[0].describe_attribute(
    Attribute='instanceType'
)

In [31]:
response['InstanceId']

'i-065e425bd2e496e5f'

In [7]:
from botocore.exceptions import ClientError
def create_cluster(client, name, key_name):
    """Creates an emr cluster
    Hardcoded ReleaseLabel: Version of EMR release, change if needed
    Hardcoded Instances: InstanceGroups: Name, Market, InstanceRole, InstanceType, InstanceCount, EBSconfiguration """

    try:
        response = client.run_job_flow(
            Name=name,
            ReleaseLabel='emr-6.7.0',
            Instances={
                'InstanceGroups': [
                    {
                        'Name': 'master',
                        'Market': 'SPOT',
                        'InstanceRole': 'MASTER',
                        'InstanceType': 'm5.xlarge',
                        'InstanceCount': 1,
                        'EbsConfiguration': {
                            'EbsBlockDeviceConfigs': [
                                {
                                    'VolumeSpecification': {
                                        'VolumeType': 'gp2',
                                        'SizeInGB': 32,
                                        
                                    },
                                    'VolumesPerInstance': 2
                                },
                            ]
                        },
                    },
                    {
                        'Name': 'core',
                        'Market': 'SPOT',
                        'InstanceRole': 'CORE',
                        'InstanceType': 'm5.xlarge',
                        'InstanceCount': 2,
                        'EbsConfiguration': {
                            'EbsBlockDeviceConfigs': [
                                {
                                    'VolumeSpecification': {
                                        'VolumeType': 'gp2',
                                        'SizeInGB': 32,
                                        
                                    },
                                    'VolumesPerInstance': 2
                                },
                            ]
                        },
                    }
                ],
                'Ec2KeyName': key_name,
                'KeepJobFlowAliveWhenNoSteps': False,
                'TerminationProtected': False
            },
            JobFlowRole='EMR_EC2_DefaultRole',
            ServiceRole='EMR_Service_Role',
            Applications=[
                {
                    'Name': 'spark',
                    
                    
                },
            ],
            
            
            
            
            ScaleDownBehavior='TERMINATE_AT_TASK_COMPLETION'
        )
    except ClientError:
        raise
    else:
        return response

In [17]:
emr = boto3.client('emr')

cluster = create_cluster(emr, 'test', 'ec2_emr')

In [8]:
redshift = boto3.client('redshift')

In [24]:
from botocore.exceptions import ClientError
def create_redshift_cluster(client, db_name, cluster_identifier, cluster_type, node_type, master_username, master_password, iam_role_arn):
    """Creates an AWS Redshift Cluster"""
    try:
        cluster = client.create_cluster(
            DBName=db_name,
            ClusterIdentifier=cluster_identifier,
            ClusterType=cluster_type,
            NodeType=node_type,
            MasterUsername=master_username,
            MasterUserPassword=master_password,  
            AutomatedSnapshotRetentionPeriod=0,
            ManualSnapshotRetentionPeriod=1,
            Port=5739,
            IamRoles=[
                iam_role_arn
            ],          
            PubliclyAccessible=True,
            DefaultIamRoleArn=iam_role_arn,
        )
        
    
    except ClientError:
        raise
    else:
        return cluster

In [19]:
cluster_identifier = 'redshift-project-1'
cluster_type = 'single-node'
node_type = 'dc2.large'
redshift_cluster = create_redshift_cluster(client=redshift, db_name='test_db'  ,cluster_identifier=cluster_identifier, cluster_type=cluster_type, 
        node_type=node_type, master_username='redshift-user-project-1', master_password='Redshift-project-1', iam_role_arn='arn:aws:iam::130747997504:role/redshift-role' )

In [3]:
sql_statements = (open("./redshiftsetup/setup.sql", 'r')).read().split(';')


In [8]:
sql_statements[3]

'\nDROP TABLE IF EXISTS spectrum.classified_movie_review'

In [21]:
redshift_data_client = boto3.client('redshift-data')

In [25]:
def run_sql_commands(client, cluster_identifier, sql_script, db_user):
    try:
        response = client.batch_execute_statement(
            ClusterIdentifier=cluster_identifier,
            Database='test_db',
            DbUser=db_user,
            Sqls=[
                sql_script
            ],
        )
    except ClientError:
        raise
    

In [26]:
run_sql_commands(client=redshift_data_client, cluster_identifier=cluster_identifier, sql_script=sql_statements, db_user='redshift-user-project-1')

In [10]:
from fabric.transfer import Transfer
from fabric.connection import Connection

In [11]:
connect = Connection(host='13.127.102.224')

In [12]:
connect.put('./boto_functions/s3functions.py', '/test-ssh')

AuthenticationException: Authentication failed.

In [14]:
ec2_connect = boto3.client('ec2-instance-connect')

In [15]:
connect_response = ec2_connect.send_ssh_public_key(
    InstanceId='i-07d14cd70b99bc049',
    InstanceOSUser='avinash_singh',
    SSHPublicKey='ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC3FlHqj2eqCdrGHuA6dRjfZXQ4HX5lXEIRHaNbxEwE5Te7xNF7StwhrDtiV7IdT5fDqbRyGw/szPj3xGkNTVoElCZ2dDFb2qYZ1WLIpZwj/UhO9l2mgfjR56UojjQut5Jvn2KZ1OcyrNO0J83kCaJCV7JoVbXY79FBMUccYNY45zmv9+1FMCfY6i2jdIhwR6+yLk8oubL8lIPyq7X+6b9S0yKCkB7Peml1DvghlybpAIUrC9vofHt6XP4V1i0bImw1IlljQS+DUmULRFSccATDscCX9ajnj7Crhm0HAZC0tBPXpFdHkPwL3yzYo546SCS9LKEwz62ymxxbL9k7h09t',
    
)